# sprint13 ディープラーニング フレームワーク1

## 1.このSprintについて

**Sprintの目的**
- フレームワークのコードを読めるようにする
- フレームワークを習得し続けられるようになる
- 理論を知っている範囲をフレームワークで動かす

**どのように学ぶか**

TensorFLowのサンプルコードを元に、これまで扱ってきたデータセットを学習していきます。

# 2.コードリーディング

TensorFlowによって2値分類を行うサンプルコードを載せました。今回はこれをベースにして進めます。


tf.keras や tf.estimator などの高レベルAPIは使用していません。低レベルなところから見ていくことにします。



## 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。


（例）


- 重みを初期化する必要があった
- エポックのループが必要だった

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

**回答**

- 学習率の設定
- バッチサイズの指定
- エポック数の設定
- 活性化関数の指定
- 重みとバイアスの初期値の設定と更新
- 最適化手法の選択


> データセットの用意

以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。

[Iris Species](https://www.kaggle.com/uciml/iris/data)


目的変数はSpeciesですが、3種類ある中から以下の2種類のみを取り出して使用します。


Iris-versicolor

Iris-virginica

In [1]:
import tensorflow as tf

In [2]:
# Google Colab使う場合
# !pip install tensorflow==1.14.0

## 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。


それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。


**《サンプルコード》**


＊TensorFlow バージョン 1.5 から 1.14 までで動作を確認済みです。

In [3]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import backend as K 

K.clear_session()

# データセットの読み込み
df = pd.read_csv("Iris.csv")
# df = pd.read_csv("drive/My Drive/Data/Iris.csv") # Google Colab使う場合
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
# NumPy 配列に変換
X = np.array(X)
y = np.array(y)
# ラベルを数値に変換
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    tf.random.set_random_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    
     # summaryの設定
    tf.summary.scalar('cross_entropy', loss_op)
    summary_op = tf.summary.merge_all()
    summary_writer = tf.summary.FileWriter('data', graph=sess.graph)
    
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        
        #tf.summary.FileWriter('iris_sigmoid', sess.graph)
        summary_str = sess.run(summary_op, feed_dict={X: X_val, Y: y_val})
        summary_writer.add_summary(summary_str, epoch)
        
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    
    # tensorboardでスカラを表示させるために必要
    summary_writer.flush()
    
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Using TensorFlow backend.


Epoch 0, loss : 1.2881, val_loss : 11.5094, acc : 0.250
Epoch 1, loss : 0.9409, val_loss : 8.2393, acc : 0.312
Epoch 2, loss : 0.6356, val_loss : 4.2207, acc : 0.500
Epoch 3, loss : 0.4789, val_loss : 3.2871, acc : 0.562
Epoch 4, loss : 0.3392, val_loss : 3.1535, acc : 0.500
Epoch 5, loss : 0.2711, val_loss : 2.3081, acc : 0.562
Epoch 6, loss : 0.2203, val_loss : 1.8365, acc : 0.688
Epoch 7, loss : 0.1893, val_loss : 1.5971, acc : 0.750
Epoch 8, loss : 0.1618, val_loss : 1.2165, acc : 0.750
Epoch 9, loss : 0.1363, val_loss : 0.9150, acc : 0.812
Epoch 10, loss : 0.1162, val_loss : 0.7208, acc : 0.812
Epoch 11, loss : 0.0979, val_loss : 0.5280, acc : 0.812
Epoch 12, loss : 0.0823, val_loss : 0.3792, acc : 0.875
Epoch 13, loss : 0.0717, val_loss : 0.3089, acc : 0.875
Epoch 14, loss : 0.0633, val_loss : 0.2742, acc : 0.875
Epoch 15, loss : 0.0579, val_loss : 0.2441, acc : 0.875
Epoch 16, loss : 0.0537, val_loss : 0.2137, acc : 0.875
Epoch 17, loss : 0.0500, val_loss : 0.1857, acc : 0.938
E

**回答**

- 学習率の設定 -> 63行目learning_rate = 0.001
- バッチサイズの指定 -> 64行目batch_size = 10
- エポック数の設定 -> 65行目num_epochs = 100
- 活性化関数の指定 -> 110行目 init = tf.global_variables_initializer()
- 重みとバイアスの初期値の設定と更新 -> 設定:82~91行目weights = {..., | biases = {...
                                  更新:104行目 train_op = optimizer.minimize(loss_op)
- 最適化手法の選択 -> 103行目 optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)"""以下を実行すると、TensorBoard 1.14.0 at http://****.local:8010/ (Press CTRL+C to quit) のようにlocalhostのurlが表示されるので、
そちらを開くと、新しいブラウザのタブが開き、TensorBoardが表示されます
"""

!tensorboard --logdir=data --port=8010

In [4]:
# """以下を実行すると、TensorBoard 1.14.0 at http://****.local:8010/ (Press CTRL+C to quit) のようにlocalhostのurlが表示されるので、
# そちらを開くと、新しいブラウザのタブが開き、TensorBoardが表示されます
# """

# !tensorboard --logdir=data --port=8010

## 3.他のデータセットへの適用

これまで扱ってきた小さなデータセットが他にもいくつかあります。上記サンプルコードを書き換え、これらに対して学習・推定を行うニューラルネットワークを作成してください。


- Iris（3種類全ての目的変数を使用）
- House Prices

どのデータセットも train, val, test の3種類に分けて使用してください。

## 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。


[Iris Species](https://www.kaggle.com/uciml/iris/data)


2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。


**《ヒント》**


以下の2箇所は2クラス分類特有の処理です。

loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))


メソッドは以下のように公式ドキュメントを確認してください。


[tf.nn.sigmoid_cross_entropy_with_logits  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits)


[tf.math.sign  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/math/sign)


＊tf.sign と tf.math.sign は同じ関数です。

In [5]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

In [6]:
# データセットの読み込み
df = pd.read_csv("Iris.csv")

# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

# NumPy 配列に変換
y = np.array(y)
X = np.array(X)

# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
# y = y.astype(np.int64)[:, np.newaxis]
y.shape

(150,)

In [7]:
# ワンホット処理
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_one_hot = enc.fit_transform(y[:, np.newaxis])
y_one_hot.shape                            

(150, 3)

In [8]:
# y_one_hot

In [9]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print('X_train.shape', X_train.shape)
print('X_val.shape', X_val.shape)
print('y_train', y_train.shape)
print('y_val.shape', y_val.shape)

X_train.shape (96, 4)
X_val.shape (24, 4)
y_train (96, 3)
y_val.shape (24, 3)


In [10]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [11]:
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
# n_classes = 1 # 2値分類用
n_classes = 3

In [12]:
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

In [13]:
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [14]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    3クラス以上の分類
    """
    tf.random.set_random_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

(参考記事)[TensorFlowによる精度計算の流れを追う](https://testpy.hatenablog.com/entry/2016/11/27/035033)
- tf.argmax(Y, 1)):行ごとに最大となる列を返す
- tf.equal():渡された２つのベクトルが一致しているか否かを見る
- tf.cast():第1パラメーターを第2パラメーターのデータ・タイプに変換する
- tf.reduce_mean():np.mean()と同じで平均を計算する

In [15]:
# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
# loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits)) # 2値分類用
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5)) # 2値分類用
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [16]:
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    
#      # summaryの設定
#     tf.summary.scalar('cross_entropy', loss_op)
#     summary_op = tf.summary.merge_all()
#     summary_writer = tf.summary.FileWriter('data', graph=sess.graph)
    
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        
#         #tf.summary.FileWriter('iris_sigmoid', sess.graph)
#         summary_str = sess.run(summary_op, feed_dict={X: X_val, Y: y_val})
#         summary_writer.add_summary(summary_str, epoch)
        
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    
#     # tensorboardでスカラを表示させるために必要
#     summary_writer.flush()
    
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 8.3052, val_loss : 63.1733, acc : 0.417
Epoch 1, loss : 4.5197, val_loss : 29.5834, acc : 0.125
Epoch 2, loss : 1.5820, val_loss : 11.5705, acc : 0.542
Epoch 3, loss : 0.6664, val_loss : 10.0250, acc : 0.667
Epoch 4, loss : 0.5632, val_loss : 8.7899, acc : 0.667
Epoch 5, loss : 0.4968, val_loss : 8.5493, acc : 0.625
Epoch 6, loss : 0.4111, val_loss : 7.2580, acc : 0.625
Epoch 7, loss : 0.3413, val_loss : 5.8605, acc : 0.625
Epoch 8, loss : 0.2782, val_loss : 4.9622, acc : 0.625
Epoch 9, loss : 0.2183, val_loss : 3.6300, acc : 0.625
Epoch 10, loss : 0.1937, val_loss : 3.4547, acc : 0.625
Epoch 11, loss : 0.1337, val_loss : 2.3338, acc : 0.667
Epoch 12, loss : 0.1114, val_loss : 1.7309, acc : 0.750
Epoch 13, loss : 0.0928, val_loss : 1.2694, acc : 0.750
Epoch 14, loss : 0.0830, val_loss : 1.1476, acc : 0.875
Epoch 15, loss : 0.0684, val_loss : 0.9816, acc : 0.917
Epoch 16, loss : 0.0608, val_loss : 0.9326, acc : 0.917
Epoch 17, loss : 0.0523, val_loss : 0.8961, acc : 0.91

## 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。


[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。


分類問題と回帰問題の違いを考慮してください。

In [29]:
# データセットの読み込み
df = pd.read_csv('train.csv')
# データフレームから条件抽出
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]

# NumPy 配列に変換
y = np.array(y)
X = np.array(X)
y = y[:, np.newaxis] # (n_samples, 1)
y.shape

(1460, 1)

In [30]:
# yに対して対数変換を行う

y = np.log(y)
y

array([[12.24769432],
       [12.10901093],
       [12.31716669],
       ...,
       [12.49312952],
       [11.86446223],
       [11.90158345]])

In [31]:
# 訓練データ80%、検証データ20%用に分割

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1168, 2)
(292, 2)
(1168, 1)
(292, 1)


In [32]:
# 標準化を行う
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

In [33]:
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train_std, y_train, test_size=0.2, random_state=0)
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(934, 2)
(234, 2)
(934, 1)
(234, 1)


In [34]:
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# n_classes = 3 # 3値以上分類用

In [35]:
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

In [36]:
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [37]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    回帰問題
    """
    tf.random.set_random_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

In [38]:
# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
# loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits)) # 2値分類用
# loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits)) # 3値以上分類用
loss_op = tf.losses.mean_squared_error(Y, logits) # 平均二乗誤差（Mean Squared Error, MSE）

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5)) # 2値分類用
# correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1)) # 3値以上分類用
correct_pred = logits

# 指標値計算
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) # 分類問題用
accuracy = tf.reduce_mean(tf.square(correct_pred - Y)) # RMSE

# variableの初期化
init = tf.global_variables_initializer()

In [39]:
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)    

    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
     
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
      
    test_acc = sess.run(accuracy, feed_dict={X: X_test_std, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 11.4022, val_loss : 19.7024, acc : 19.702
Epoch 1, loss : 1.0593, val_loss : 6.8762, acc : 6.876
Epoch 2, loss : 0.5319, val_loss : 4.3998, acc : 4.400
Epoch 3, loss : 0.3653, val_loss : 3.2725, acc : 3.272
Epoch 4, loss : 0.2800, val_loss : 2.6316, acc : 2.632
Epoch 5, loss : 0.2280, val_loss : 2.2450, acc : 2.245
Epoch 6, loss : 0.1927, val_loss : 1.9976, acc : 1.998
Epoch 7, loss : 0.1668, val_loss : 1.8074, acc : 1.807
Epoch 8, loss : 0.1458, val_loss : 1.6507, acc : 1.651
Epoch 9, loss : 0.1277, val_loss : 1.5150, acc : 1.515
Epoch 10, loss : 0.1127, val_loss : 1.3961, acc : 1.396
Epoch 11, loss : 0.1000, val_loss : 1.2936, acc : 1.294
Epoch 12, loss : 0.0900, val_loss : 1.2067, acc : 1.207
Epoch 13, loss : 0.0817, val_loss : 1.1486, acc : 1.149
Epoch 14, loss : 0.0750, val_loss : 1.0893, acc : 1.089
Epoch 15, loss : 0.0694, val_loss : 1.0298, acc : 1.030
Epoch 16, loss : 0.0649, val_loss : 0.9770, acc : 0.977
Epoch 17, loss : 0.0610, val_loss : 0.9255, acc : 0.925

## 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。

In [ ]:
# Google Colab使う場合
# !pip install keras==2.2.4

In [40]:
# MNISTのデータセットを読み込み
import numpy as np
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(X_train.shape) # (60000, 28, 28)
print(X_test.shape) # (10000, 28, 28)
print(X_train[0].dtype) # uint8
print(X_train[0])

11493376/11490434 [==============================] - 3s 0us/step
(60000, 28, 28)
(10000, 28, 28)
uint8
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   9

In [41]:
# 前処理
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0

1.0
0.0


In [45]:
# 平滑化
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
print(X_train.shape) # (60000, 784)
print(X_test.shape) # (10000, 784)

(60000, 784)
(10000, 784)


In [47]:
# ワンホット処理
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.fit_transform(y_test[:, np.newaxis])
print(y_train_one_hot.shape) # (60000, 10)
print(y_test_one_hot.shape) # (10000, 10)

(60000, 10)
(10000, 10)


In [48]:
# trainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2, random_state=0)
print('X_train.shape', X_train.shape)
print('X_val.shape', X_val.shape)
print('y_train', y_train.shape)
print('y_val.shape', y_val.shape)

X_train.shape (48000, 784)
X_val.shape (12000, 784)
y_train (48000, 10)
y_val.shape (12000, 10)


In [55]:
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

In [50]:
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

In [51]:
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [52]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    3クラス以上の分類
    """
    tf.random.set_random_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

In [53]:
# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
# loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits)) # 2値分類用
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5)) # 2値分類用
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(tf.nn.softmax(logits), axis=1)) # 最大値のインデックスを比較

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

In [56]:
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 2.6046, val_loss : 7.2459, acc : 0.773
Epoch 1, loss : 0.4067, val_loss : 2.5514, acc : 0.756
Epoch 2, loss : 0.1471, val_loss : 1.2032, acc : 0.741
Epoch 3, loss : 0.0866, val_loss : 0.9010, acc : 0.796
Epoch 4, loss : 0.0660, val_loss : 0.7807, acc : 0.832
Epoch 5, loss : 0.0553, val_loss : 0.7139, acc : 0.844
Epoch 6, loss : 0.0470, val_loss : 0.6484, acc : 0.871
Epoch 7, loss : 0.0418, val_loss : 0.6069, acc : 0.886
Epoch 8, loss : 0.0378, val_loss : 0.5729, acc : 0.894
Epoch 9, loss : 0.0345, val_loss : 0.5525, acc : 0.899
test_acc : 0.897


(参考記事)[TensorFlow入門 - 四則演算と基礎的な数学関数まとめ](https://qiita.com/negabaro/items/bdaeaf6ba4bc9fc81208)